# Integrantes

- Sergio Orellana 221122
- Rodrigo Mansilla 22611
- Carlos Valladares 221164

# Ejercicio 1

## Simulación de tiempo discreto (síncrona) vs. tiempo continuo (asíncrona)

- **Tiempo discreto (síncrona)**: yo actualizo el estado del sistema en pasos fijos (ticks) con un reloj global, de modo que todos los componentes se actualizan “a la vez” en cada paso. Esto es análogo a trabajar con señales muestreadas y diferencia-ecuaciones en control digital. 

- **Tiempo continuo (asíncrona)**: yo permito cambios en cualquier instante real; típicamente lo implemento como simulación dirigida por eventos que “salta” al próximo evento en una cola de eventos, sin pasos fijos. (En marcos como SimPy, los procesos se programan con env.timeout y el motor avanza al siguiente evento).

**(a) ¿Cuándo prefiero cada una?**
Prefiero tiempo discreto (síncrona) cuando:
- El fenómeno ya está muestreado por naturaleza (p. ej., control digital, DSP, comunicaciones digitales). 
- Necesito simplicidad y determinismo (flujo secuencial más fácil de razonar y depurar). 

Prefiero tiempo continuo (asíncrona) cuando:
- El timing exacto de los sucesos importa (colas/redes, tráfico, servicios), porque el motor salta entre eventos sin simular “tiempo vacío”. 
- Busco escala y capacidad de respuesta en sistemas dirigidos por eventos o procesamiento asíncrono. 

**(b) Compensaciones computacionales (velocidad vs. precisión)**
- **Discreto/síncrona:** suele ser más rápida por implementación y más simple de depurar, pero su precisión depende del tamaño del paso (riesgo de aliasing/artefactos si el paso es grande). 
- **Continuo/asíncrona:** logra mejor precisión temporal (eventos en el instante exacto) y puede ser eficiente al evitar pasos donde nada ocurre; a cambio, incrementa la complejidad (concurrencia, manejo de callbacks/eventos, depuración).

## Mecanismos de gestión de eventos
**(a) ¿Cómo gestionan los cambios de estado las colas de eventos?**

Cuando modelo de forma asíncrona, mantengo una cola de eventos con elementos del tipo (tiempo, tipo, datos). El ciclo de simulación hace:

- Extraer el siguiente evento (el de menor marca de tiempo).
- Avanzar el reloj de simulación a ese instante.
- Aplicar las transiciones de estado y programar nuevos eventos (insertarlos en la cola).

Este patrón aparece tanto en frameworks de simulación (p. ej., SimPy con procesos/env.timeout) como en bucles de eventos (JS: el event loop despacha tareas desde la event queue cuando la pila está libre). 

En arquitectura de sistemas, las message queues/colas de mensajes permiten desacoplar productores y consumidores de eventos, reforzando la idea de que los cambios de estado se encolan y se procesan sin bloquear el flujo principal

**(b) Gestión de prioridades para eventos concurrentes**

Para resolver eventos “al mismo tiempo” o con diferentes importancias, utilizo una cola de prioridad (típicamente un binary heap), donde sale primero el evento de mayor prioridad (o menor tiempo y, si hay empates, un criterio secundario). Las operaciones de insertar y extraer tienen costo O(log n), lo que mantiene eficiente el motor de simulación. Además, las simulaciones dirigidas por eventos son un caso de uso clásico de colas de prioridad. 

Como ejemplo de priorización de colas en un bucle de eventos, en JavaScript existen microtareas que se procesan antes de las tareas normales, mostrando cómo la política de prioridad afecta qué se ejecuta primero cuando hay concurrencia.



# Referencias

- GeeksforGeeks. (2024, February 27). Continuous time and discrete time control systems. GeeksforGeeks. https://www.geeksforgeeks.org/electrical-engineering/continuous-time-and-discrete-time-control-systems/
- GeeksforGeeks. (2024b, May 31). Synchronous and asynchronous programming. GeeksforGeeks. https://www.geeksforgeeks.org/javascript/synchronous-and-asynchronous-programming/
- GeeksforGeeks. (2020, November 19). Basics of Discrete Event Simulation using SimPy. GeeksforGeeks. https://www.geeksforgeeks.org/python/basics-of-discrete-event-simulation-using-simpy/
- GeeksforGeeks. (2025, July 23). EventDriven Architecture System design. GeeksforGeeks. https://www.geeksforgeeks.org/system-design/event-driven-architecture-system-design
- GeeksforGeeks. (2025a, July 23). Asynchronous processing in system design. GeeksforGeeks. https://www.geeksforgeeks.org/system-design/asynchronous-processing-in-system-design/
- GeeksforGeeks. (2025c, August 5). Event queue in JavaScript. GeeksforGeeks. https://www.geeksforgeeks.org/javascript/event-queue-in-javascript
- GeeksforGeeks. (2025d, August 7). Message queues system design. GeeksforGeeks. https://www.geeksforgeeks.org/system-design/message-queues-system-design
- GeeksforGeeks. (2025c, July 23). What is Priority Queue | Introduction to Priority Queue. GeeksforGeeks. https://www.geeksforgeeks.org/dsa/priority-queue-set-1-introduction/
- GeeksforGeeks. (2025a, July 23). Applications of priority queue. GeeksforGeeks. https://www.geeksforgeeks.org/dsa/applications-priority-queue/
- GeeksforGeeks. (2023, January 25). What are the microtask and macrotask within an event loop in JavaScript ? GeeksforGeeks. https://www.geeksforgeeks.org/javascript/what-are-the-microtask-and-macrotask-within-an-event-loop-in-javascript/